# Aprendizaje Automático 2
Tecnicatura Universitaria en Inteligencia Artificial

Materia: IA5.1 Aprendizaje Automático 2

**Trabajo Práctico 2: Redes Recurrentes**

**Problema 2 - Fake News**


**Docentes:**
- Matias Moreyra
- Bruno Cocitto
- Facundo Moreyra


**Integrantes:**

| Apellido y Nombre | Legajo   |
| ----------------- | -------- |
| Aguirre, Fabián   | A-4516/1 |
| Fontela, Facundo  | F-3724/9 |
| Wagner, Juan      | W-0557/6 |

Año: 2024


In [ ]:
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, LSTM, Reshape
from keras import models
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
import time

In [ ]:
%pip install pydub

# 2) Problema 2 - Fake News

**Descripción:**
En el siguiente problema, se presenta un conjunto de datos correspondientes a resúmenes de artículos de noticias. El objetivo del problema es crear un modelo capaz de generar resúmenes ficticios de forma aleatoria.

**Dataset:**
https://www.tensorflow.org/datasets/catalog/ag_news_subset


El dataset proporcionado incluye 120000 artículos correspondientes a 4 categorías distintas. En este caso, la categoría no es relevante, sólo utilizaremos el dataset como un cuerpo de texto para entrenar un modelo recurrente de generación de texto.

**Objetivo:**
Utilizando el dataset construido, el objetivo es construir modelos de generación de texto utilizando redes neuronales que puedan generar noticias ficticias. Consultar el siguiente lab como referencia para la implementación:


https://colab.research.google.com/github/FCEIA-AAII/lab10/blob/master/lab10-b.ipynb


Se solicita experimentar con los siguientes tipos de modelos:

**Caracter a caracter:** entrenar un modelo de generación de texto a nivel de caracteres como el correspondiente al Lab10 mencionado anteriormente.

**Palabra a palabra:** entrenar un modelo de generación de texto a nivel de palabras, adecuando los procesos de entrenamiento e inferencia según sea necesario.

Generar artículos al azar y seleccionar 5 para cada modelo que resulten de interés. Comparar cualitativamente el tipo de resultado que se obtiene para cada tipo de modelo.


No se requiere un análisis de métricas para este problema, se espera un análisis cualitativo de los resultados obtenidos.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [ ]:
dataset = tfds.load('ag_news_subset', as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.F4RJ5L_1.0.0/ag_news_subset-train.tfrecord*...: …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.F4RJ5L_1.0.0/ag_news_subset-test.tfrecord*...:  …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


AG es una colección de más de 1 millón de artículos de noticias. El conjunto de datos de clasificación de temas de noticias de AG se construye eligiendo las 4 clases más grandes del corpus original. Cada clase contiene 30.000 muestras de entrenamiento y 1.900 muestras de prueba. El número total de muestras de entrenamiento es de 120.000 y de pruebas de 7.600.

In [ ]:
train_ds, test_ds = dataset['train'], dataset['test']

# Obtenemos todos los textos y etiquetas del dataset
def get_all_texts(ds):
    texts = []
    for text, _ in ds:
        texts.append(text.numpy().decode('utf-8'))
    return ' '.join(texts)

# Obtenemos los textos del conjunto de entrenamiento en una sola variable
train_text = get_all_texts(train_ds)
test_texts = get_all_texts(test_ds)

print(f'Longitud del texto concatenado del conjunto de entrenamiento: {len(train_text)}')

print(train_text[:1000])

Longitud del texto concatenado del conjunto de entrenamiento: 23328241
AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions. Reuters - Major League Baseball\Monday announced a decision on the appeal filed by Chicago Cubs\pitcher Kerry Wood regarding a suspension stemming from an\incident earlier this season. President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say. Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger. London, England (Sports Network) - England midfielder Steven Gerrard injured his groin late in Thursday #39;s training session, but is hopeful he will be ready for Saturday #39;s World Cup qualifier against Austria. TOKYO - Sony Corp. is bankin

In [ ]:
#Utilizamos solo el texto de entrenamiento
text = train_text

In [ ]:
text = train_text[:1000]

In [ ]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

82 unique characters


In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids = all_ids.numpy()

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, includi'
b'ng databases, Web services, and financial transactions. Reuters - Major League Baseball\\Monday announ'
b'ced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming'
b' from an\\incident earlier this season. President Bush #39;s  quot;revenue-neutral quot; tax reform ne'
b'eds losers to balance its winners, and people claiming the federal deduction for state and local taxe'


In [ ]:
#Funcion que toma una secuencia como entrada, la duplica y la desplaza para alinear la entrada y la etiqueta para cada timestep
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, includ'
Target: b'MD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, includi'


In [ ]:
# Se divide el dataset en entrenamiento y 10% para validación
val_size = int(0.1 * len(all_ids) / seq_length)
train_ds = dataset.skip(val_size)
val_ds = dataset.take(val_size)

In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
val_ds = val_ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 1024

In [ ]:
#Se define el modelo y se utilizan dropout y regularización L2 para que no haya sobreajuste

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform',
                                   dropout=0.2,
                                   recurrent_regularizer=tf.keras.regularizers.l2(0.01))
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
#Se define la pérdida y el optimizador
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
# Checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
# Early Stopping
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
#Se compila el modelo con la pérdida y el optimizador definido anteriormente
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
EPOCHS = 10

In [ ]:
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[checkpoint_callback, early_stopping_callback])

Epoch 1/10
3244/3244 [==============================] - 204s 60ms/step - loss: 2.0573 - val_loss: 1.7440
Epoch 2/10
3244/3244 [==============================] - 202s 61ms/step - loss: 1.7120 - val_loss: 1.6648
Epoch 3/10
3244/3244 [==============================] - 204s 61ms/step - loss: 1.6625 - val_loss: 1.6350
Epoch 4/10
3244/3244 [==============================] - 204s 61ms/step - loss: 1.6390 - val_loss: 1.6165
Epoch 5/10
3244/3244 [==============================] - 202s 61ms/step - loss: 1.6247 - val_loss: 1.6100
Epoch 6/10
3244/3244 [==============================] - 200s 60ms/step - loss: 1.6152 - val_loss: 1.6000
Epoch 7/10
3244/3244 [==============================] - 202s 60ms/step - loss: 1.6078 - val_loss: 1.5916
Epoch 8/10
3244/3244 [==============================] - 202s 60ms/step - loss: 1.6021 - val_loss: 1.5847
Epoch 9/10
3244/3244 [==============================] - 201s 60ms/step - loss: 1.5973 - val_loss: 1.5818
Epoch 10/10
3244/3244 [==============================] 

Vemos que se podría aumentar las epocas para disminuir la pérdida pero este modelo tarda demasiado en entrenar y nos quedamos sin GPU.

In [ ]:
#Funcion bucle para generar los caracteres

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    predicted_logits = predicted_logits + self.prediction_mask

    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [ ]:
# Se crea el modelo de un paso
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
#Se generan 5 alternativas de texto
start = time.time()
states = None
next_char = tf.constant(['The news', 'The news', 'The news', 'The news', 'The news'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'The news condustry on Tuesday, whethere at the Eurio Internet wheen touchnation to to with a show on tumber, a crosence to suse of the goold run.. Tex Wednesday in GERLIO (Sponsionsheer governments are company Co. hoppice in Iran partfulter; cheens open. Burinai, buying troady this week. Iraq in continu in the the of Drerused in crudictiven due St. LOS, November New Despite hand Bill Staudy sit back technomester horse taking dised for the said the York of consumalty. Reuters) - Thousady about of orthanks former in applican Luisine Rognest Mari Docrasive Broad Plues and Cong Bush Finger-baded a nativer Declar to can One to  and in a low-boung. Michiling semen matches key  start forwidges their lossing of servals of the losseguatives opene in long, breaking coverate in MG6th dise forceedit by first annoy to usely in propean wilses to company of the service Palesters. The all-corrway night a firmed in a guilty today with their locates for a yesterday. Oakland contoring China

# b)

In [ ]:
# Volvemos cargar el dataset para tener data limpia
dataset = tfds.load('ag_news_subset', as_supervised=True)
train_ds, test_ds = dataset['train'], dataset['test']

train_text = get_all_texts(train_ds)
test_texts = get_all_texts(test_ds)

text = train_text

In [ ]:
# Por problemas de performance reducimos el texto
text = text[:5000000]

In [ ]:
# Tokenizar el texto en palabras
words = text.split()
vocab = sorted(set(words))
print(f'{len(vocab)} unique words')

68339 unique words


In [ ]:
# Crear las funciones de lookup para palabras
ids_from_words = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
words_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')

all_ids = ids_from_words(words)
all_ids = all_ids.numpy()
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 20

sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
# Dividir el dataset en entrenamiento y validación
val_size = int(0.1 * len(all_ids) / seq_length)  # 10% de validación
train_ds = dataset.skip(val_size)
val_ds = dataset.take(val_size)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

train_ds = (
    train_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

val_ds = val_ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

vocab_size = len(ids_from_words.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

In [ ]:
# Se utiliza el mismo modelo que en la parte a
model = MyModel(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)

In [ ]:
# Checkpoints
checkpoint_dir = './training_checkpoints_b'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
# Compilar el modelo
model.compile(optimizer=optimizer, loss=loss)

In [ ]:
EPOCHS=10
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=[checkpoint_callback, early_stopping_callback])
# El modelo parece ajustarse a los de datos de entrenamiento, reduciendo el loss, pero con respecto a los datos de validacion mantiene la misma perdida

Epoch 1/10
531/531 [==============================] - 118s 214ms/step - loss: 8.1940 - val_loss: 7.1442
Epoch 2/10
531/531 [==============================] - 118s 222ms/step - loss: 6.6294 - val_loss: 6.7723
Epoch 3/10
531/531 [==============================] - 131s 246ms/step - loss: 5.9750 - val_loss: 6.6169
Epoch 4/10
531/531 [==============================] - 136s 254ms/step - loss: 5.3862 - val_loss: 6.6045
Epoch 5/10
531/531 [==============================] - 140s 263ms/step - loss: 4.8030 - val_loss: 6.7154
Epoch 6/10
531/531 [==============================] - 128s 239ms/step - loss: 4.2240 - val_loss: 6.9035
Epoch 7/10
531/531 [==============================] - 129s 242ms/step - loss: 3.6879 - val_loss: 7.1066


In [ ]:
# Utilizamos el mismo modelo one step, con el modelo fiteado al dataset de palabras
one_step_model = OneStep(model, words_from_ids, ids_from_words)

In [ ]:
# Generación de texto palabra a palabra
# En los resultados se puede observar como el modelo elige palabras con muy poca coherencia
# Al modelo le faltan gran cantidad de epocas de entrenamiento para poder generar frases con sentido.
start = time.time()
states = None
next_word = tf.constant(['The news'])
result = [next_word]

for n in range(99):
    next_word, states = one_step_model.generate_one_step(next_word, states=states)
    result.append(next_word)

result = tf.strings.reduce_join(result, separator=' ', axis=-1)
python_list = result.numpy().tolist()
decoded_list = [item.decode('utf-8') for item in python_list]
full_text = ' '.join(decoded_list)

end = time.time()
print(full_text, '\n\n' + '_' * 80)
print('\nRun time:', end - start)

The news answer Greg standards, the might was choice and Relations answer OAO #39;Keefe the was little last never Should #39;affaires a loss answer The but was choice and #39;affaires the and #39;affaires comments choice which have and #39;affaires underdog drives an for You're of has choice - the is fear TV Classic. weather Do must was an quot; reports the is answer Do win. SAN smacked #39;affaires decision activities the days answer XM over Rashard pictured #39;affaires immigration If is offensive and #39;affaires a strike who and #39;affaires free will #39;accs choice officials choice was movie when boxes mean from 

________________________________________________________________________________

Run time: 0.41494321823120117
